In [1]:
import os
import shutil
from pathlib import Path

import pandas as pd

In [8]:
relative_root = Path('..')
meta_parsed = pd.read_csv(relative_root.joinpath('data/interim/meta_publications_merged.csv'))
meta_ocr = pd.read_csv(relative_root.joinpath('data/interim/meta_publications_ocr.csv'))

In [15]:
destination_root = relative_root.joinpath('data/raw/publications_manual_export/UTA publications')

# files to copy from `UTA publications`
for i, row in meta_parsed.iterrows():
    # input path stays unchanged
    input_path = Path(row['path'])
    filename = input_path.name

    # copy directory structure from the UTA Publications    
    path_structure = input_path.parent.parts[3:]
    path_structure = Path(*path_structure)
    # create destination directory
    destination_dir = destination_root.joinpath(path_structure)
    destination_dir.mkdir(parents=True, exist_ok=True)
    # copy file
    destination_file_path = destination_dir.joinpath(filename)
    shutil.copy(input_path, destination_file_path)

In [21]:
# files to copy from `publications_manual_export/ocr_files_after`
input_root = relative_root.joinpath('data/raw/publications_manual_export/ocr_files_after')
destination_root = relative_root.joinpath('data/raw/publications_manual_export/UTA publications')

# files to copy from `UTA publications`
for i, row in meta_ocr.iterrows():
    # input path is used to generate destination path
    imaginary_input_path = Path(row['path'])
    # but the real input path is just the filename
    filename = imaginary_input_path.name
    input_path = input_root.joinpath(filename)

    # copy directory structure from the UTA Publications    
    path_structure = imaginary_input_path.parent.parts[3:]
    path_structure = Path(*path_structure)
    # create destination directory
    destination_dir = destination_root.joinpath(path_structure)
    destination_dir.mkdir(parents=True, exist_ok=True)
    # copy file
    destination_file_path = destination_dir.joinpath(filename)
    shutil.copy(input_path, destination_file_path)

In [24]:
# make sure no files got lost

def get_file_paths(folder_path: str) -> tuple[list, list]:
    """
    """

    pdf_paths = []
    other_paths = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            filepath = os.path.join(root, filename)
            if filename.endswith('.pdf'):
                pdf_paths.append(filepath)
            else:
                other_paths.append(filepath)

    return pdf_paths, other_paths


datadir_orig = str(relative_root.joinpath('data/raw/UTA publications'))
datadir_reco = str(relative_root.joinpath('data/raw/publications_manual_export/UTA publications'))

pdf_paths_orig, other_paths_orig = get_file_paths(datadir_orig)
pdf_paths_reco, other_paths_reco = get_file_paths(datadir_reco)

assert len(pdf_paths_orig) == len(pdf_paths_reco)


False